In [1]:
import pandas as pd
import pandera as pa    #utiliza esquemas para validar os dados 

In [2]:
# sep=";" é usado pois os dados neste arquivo é separado por ';'
# parse_dates= usado para que a coluna que contém datas seja entendida como datas
# dayfirst=True para deixar aaaa-mm-dd
# na_values= substituira os valores_ausentes por na
valores_ausentes = ['**', '***', '###!', '####', '****', '*****', 'NULL']
df = pd.read_csv("ocorrencia.csv", sep=";", encoding='latin1', parse_dates=['ocorrencia_dia'], dayfirst=True, na_values=valores_ausentes)
df.head(10)

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
0,81027,81027,INCIDENTE GRAVE,SÃO PAULO,SP,SBSP,2022-10-09,16:30:00,0
1,81030,81030,INCIDENTE,VITÓRIA,ES,SBVT,2022-10-09,16:20:00,0
2,81023,81023,INCIDENTE GRAVE,AMERICANA,SP,SDAI,2022-10-06,15:30:00,0
3,81029,81029,INCIDENTE,BELO HORIZONTE,MG,SBBH,2022-10-06,00:44:00,0
4,81025,81025,INCIDENTE,MANAUS,AM,SBEG,2022-10-05,20:56:00,0
5,81019,81019,ACIDENTE,PORTO ALEGRE,RS,NaN,2022-10-03,20:50:00,0
6,81022,81022,INCIDENTE,LOBATO,PR,NaN,2022-10-03,15:38:00,0
7,81021,81021,ACIDENTE,NOVA BANDEIRANTES,MT,NaN,2022-10-02,19:00:00,0
8,81009,81009,INCIDENTE GRAVE,NAVIRAÍ,MS,**NI,2022-10-01,10:10:00,0
9,81012,81012,INCIDENTE,LINHARES,ES,NaN,2022-09-30,12:20:00,0


In [3]:
schema = pa.DataFrameSchema(
    columns={
        #required=False a coluna pode existir ou não
        #"codigo": pa.Column(pa.Int, required=False),
     
        "codigo_ocorrencia": pa.Column(pa.Int),
        "codigo_ocorrencia2":pa.Column(pa.Int),
        "ocorrencia_classificacao": pa.Column(pa.String),
        "ocorrencia_cidade": pa.Column(pa.String),
        
        #Check.str_lenghtExpressão que dá o tamanho minimo e máximo do dado que vem na coluna
        #nullable=True aceita valores nulo
        "ocorrencia_uf": pa.Column(pa.String, pa.Check.str_length(2,2), nullable=True),
        "ocorrencia_aerodromo":	pa.Column(pa.String, nullable=True),
        "ocorrencia_dia": pa.Column(pa.DateTime),

        #Expressão regular para o formato 24hrs
        "ocorrencia_hora": pa.Column(pa.String, pa.Check.str_matches(r'^([0-1]?[0-9]|[2][0-3]):([0-5][0-9])(:[0-5][0-9])?$'), nullable=True), 
        "total_recomendacoes":pa.Column(pa.Int)
    }
)

In [4]:
schema.validate(df) 

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
0,81027,81027,INCIDENTE GRAVE,SÃO PAULO,SP,SBSP,2022-10-09,16:30:00,0
1,81030,81030,INCIDENTE,VITÓRIA,ES,SBVT,2022-10-09,16:20:00,0
2,81023,81023,INCIDENTE GRAVE,AMERICANA,SP,SDAI,2022-10-06,15:30:00,0
3,81029,81029,INCIDENTE,BELO HORIZONTE,MG,SBBH,2022-10-06,00:44:00,0
4,81025,81025,INCIDENTE,MANAUS,AM,SBEG,2022-10-05,20:56:00,0
...,...,...,...,...,...,...,...,...,...
6764,40324,40324,INCIDENTE,PELOTAS,RS,SBPK,2010-01-05,19:25:00,0
6765,39527,39527,ACIDENTE,LUCAS DO RIO VERDE,MT,NaN,2010-01-04,17:30:00,0
6766,40211,40211,INCIDENTE,RIO DE JANEIRO,RJ,NaN,2010-01-03,12:00:00,0
6767,40349,40349,INCIDENTE,BELÉM,PA,SBBE,2010-01-03,11:05:00,0


### Filtros

In [5]:
#Buscar dados de uma coluna
df['ocorrencia_uf']

0       SP
1       ES
2       SP
3       MG
4       AM
        ..
6764    RS
6765    MT
6766    RJ
6767    PA
6768    RJ
Name: ocorrencia_uf, Length: 6769, dtype: object

In [6]:
#Quantidade de valores não informados
df.isna().sum()

codigo_ocorrencia              0
codigo_ocorrencia2             0
ocorrencia_classificacao       0
ocorrencia_cidade              0
ocorrencia_uf                  4
ocorrencia_aerodromo        2417
ocorrencia_dia                 0
ocorrencia_hora                2
total_recomendacoes            0
dtype: int64

In [7]:
#Localiza dados nulos em uma coluna
df.loc[df.ocorrencia_uf.isnull()]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
283,80665,80665,ACIDENTE,NÃO IDENTIFICADA,NaN,**NI,2022-04-22,20:00:00,0
320,80631,80631,INCIDENTE,NÃO IDENTIFICADA,NaN,NaN,2022-04-04,11:15:00,0
912,79956,79956,ACIDENTE,NÃO IDENTIFICADA,NaN,**NI,2021-02-26,11:00:00,0
4541,49474,49474,ACIDENTE,ÁGUAS INTERNACIONAIS,NaN,NaN,2013-09-02,02:54:00,0


In [10]:
#Cidades com o correncias com mais de 10 recomendações
filtro = df.total_recomendacoes>10
df.loc[filtro, ['ocorrencia_cidade', 'total_recomendacoes']]

,ocorrencia_cidade,total_recomendacoes
2836,VITÓRIA,12
3970,SANTOS,13
4796,BRASÍLIA,11
5705,UBERLÂNDIA,20
5885,SÃO JOSÉ DOS PINHAIS,11
5894,RECIFE,23
6040,ITÁPOLIS,11
6238,SÃO PAULO,11
6441,RIO DE JANEIRO,19
6705,SÃO PAULO,11


In [12]:
#Ocorrencias cuja classificação == INCIDENTE GRAVE
filtro = df.ocorrencia_classificacao == 'INCIDENTE GRAVE'
df.loc[filtro, ['ocorrencia_cidade', 'total_recomendacoes']]

,ocorrencia_cidade,total_recomendacoes
0,SÃO PAULO,0
2,AMERICANA,0
8,NAVIRAÍ,0
10,GOIÂNIA,0
13,ARAÇATUBA,0
...,...,...
6733,PALMAS,0
6739,CURITIBA,2
6756,CASCAVEL,2
6757,PARÁ DE MINAS,0


In [13]:
#Ocorrencias cuja classificação == INCIDENTE GRAVE e o estado == SP
filtro1 = df.ocorrencia_classificacao == 'INCIDENTE GRAVE'
filtro2 = df.ocorrencia_uf == 'SP'
df.loc[filtro1 & filtro2]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
0,81027,81027,INCIDENTE GRAVE,SÃO PAULO,SP,SBSP,2022-10-09,16:30:00,0
2,81023,81023,INCIDENTE GRAVE,AMERICANA,SP,SDAI,2022-10-06,15:30:00,0
13,81003,81003,INCIDENTE GRAVE,ARAÇATUBA,SP,SBAU,2022-09-26,18:30:00,0
53,80951,80951,INCIDENTE GRAVE,ASSIS,SP,SNAX,2022-08-31,18:45:00,0
69,80925,80925,INCIDENTE GRAVE,SÃO PAULO,SP,SBMT,2022-08-20,19:00:00,0
...,...,...,...,...,...,...,...,...,...
6455,42323,42323,INCIDENTE GRAVE,SÃO JOSÉ DO RIO PRETO,SP,SBSR,2010-08-05,12:01:00,0
6470,42289,42289,INCIDENTE GRAVE,SÃO PAULO,SP,SBMT,2010-07-29,20:06:00,0
6548,41411,41411,INCIDENTE GRAVE,SÃO PAULO,SP,SBMT,2010-06-08,19:30:00,2
6595,40620,40620,INCIDENTE GRAVE,GUARULHOS,SP,SBGR,2010-05-06,15:50:00,0


In [15]:
#Ocorrencias cuja (classificação == INCIDENTE GRAVE OU classificação == INCIDENTE) e o estado == SP
filtro1 = df.ocorrencia_classificacao.isin(['INCIDENTE GRAVE', 'INCIDENTE'])
filtro2 = df.ocorrencia_uf == 'SP'
df.loc[filtro1 & filtro2]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
0,81027,81027,INCIDENTE GRAVE,SÃO PAULO,SP,SBSP,2022-10-09,16:30:00,0
2,81023,81023,INCIDENTE GRAVE,AMERICANA,SP,SDAI,2022-10-06,15:30:00,0
13,81003,81003,INCIDENTE GRAVE,ARAÇATUBA,SP,SBAU,2022-09-26,18:30:00,0
14,81001,81001,INCIDENTE,BRAGANÇA PAULISTA,SP,SBBP,2022-09-25,20:00:00,0
18,80997,80997,INCIDENTE,GUARULHOS,SP,SBGR,2022-09-21,11:30:00,0
...,...,...,...,...,...,...,...,...,...
6736,39848,39848,INCIDENTE,GUARULHOS,SP,SBGR,2010-01-25,17:05:00,0
6741,39768,39768,INCIDENTE,CAMPINAS,SP,NaN,2010-01-21,20:45:00,0
6743,39847,39847,INCIDENTE,SOROCABA,SP,NaN,2010-01-20,13:10:00,0
6751,39809,39809,INCIDENTE,SÃO PAULO,SP,NaN,2010-01-15,15:00:00,0


In [16]:
#Ocorrencias cuja cidade comecem com a letra C
filtro1 = df.ocorrencia_cidade.srt[0] == 'C'
df.loc[filtro1]

AttributeError: 'Series' object has no attribute 'srt'

In [ ]:
#Juntando colunas de dia e hora
df['ocorrencia_dia_hora'] = pd.to_datetime (df.ocorrencia_dia.astype(str) + ' ' + df.ocorrencia_hora)
df.dtypes

In [34]:
#Ocorrencias do ano de 2015 e mês 12 e dias entre 3 e 8 e entre 11 e 13 horas
filtro1 = df.ocorrencia_dia_hora.dt.year == 2015
filtro2 = df.ocorrencia_dia_hora.dt.month == 12
filtro3 = (df.ocorrencia_dia_hora.dt.day > 2) & (df.ocorrencia_dia_hora.dt.day < 9)
filtro4 = (df.ocorrencia_dia_hora >= '2015-12-08 11:00:00') & (df.ocorrencia_dia_hora >= '2015-12-08 13:00:00')
df.loc[filtro1 & filtro2 & filtro3 & filtro4]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes,ocorrencia_dia_hora
3327,53628,53628,ACIDENTE,AGUAÍ,SP,NaN,2015-12-08,14:30:00,1,2015-12-08 14:30:00
3329,53631,53631,INCIDENTE,CAMPINAS,SP,SBKP,2015-12-08,16:19:00,0,2015-12-08 16:19:00
3330,60636,60636,INCIDENTE,CAXIAS DO SUL,RS,SBCX,2015-12-08,13:00:00,0,2015-12-08 13:00:00


In [35]:
#Database com informações agrupadas
filtro1 = df.ocorrencia_dia_hora.dt.year == 2015
filtro2 = df.ocorrencia_dia_hora.dt.month == 12
df201512 = df.loc[filtro1 & filtro2]

In [38]:
#Quantidade de ocorrencias por cada categoria
df201512.groupby(['ocorrencia_classificacao']).size().sort_values(ascending=False)

ocorrencia_classificacao
INCIDENTE          23
ACIDENTE           13
INCIDENTE GRAVE     3
dtype: int64

In [41]:
#Dataframe com ocorrencias do 2010 no sudeste
filtro1 = df.ocorrencia_dia.dt.year == 2010
filtro2 = df.ocorrencia_uf.isin(['SP', 'MG', 'ES', 'RJ'])
dfsudeste2010 = df.loc[filtro1 & filtro2]
dfsudeste2010

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes,ocorrencia_dia_hora
6217,43730,43730,INCIDENTE,VITÓRIA,ES,NaN,2010-12-30,18:10:00,0,2010-12-30 18:10:00
6218,43975,43975,INCIDENTE,ARRAIAL DO CABO,RJ,NaN,2010-12-30,14:28:00,0,2010-12-30 14:28:00
6219,43724,43724,ACIDENTE,COSMÓPOLIS,SP,NaN,2010-12-29,12:00:00,0,2010-12-29 12:00:00
6224,43548,43548,INCIDENTE,SÃO PAULO,SP,SBSP,2010-12-26,22:10:00,0,2010-12-26 22:10:00
6226,43722,43722,ACIDENTE,CAJOBI,SP,NaN,2010-12-23,11:00:00,0,2010-12-23 11:00:00
...,...,...,...,...,...,...,...,...,...,...
6753,39828,39828,INCIDENTE,SANTOS,SP,NaN,2010-01-15,17:45:00,0,2010-01-15 17:45:00
6757,39711,39711,INCIDENTE GRAVE,PARÁ DE MINAS,MG,NaN,2010-01-10,20:00:00,0,2010-01-10 20:00:00
6759,40069,40069,ACIDENTE,SÃO PAULO,SP,SBMT,2010-01-10,14:50:00,8,2010-01-10 14:50:00
6766,40211,40211,INCIDENTE,RIO DE JANEIRO,RJ,NaN,2010-01-03,12:00:00,0,2010-01-03 12:00:00


In [42]:
#Quantidade de incidentes na região sudeste
dfsudeste2010.groupby(['ocorrencia_classificacao']).size()

ocorrencia_classificacao
ACIDENTE            39
INCIDENTE          171
INCIDENTE GRAVE     18
dtype: int64

In [46]:
#Quantidade de recomendações por cidade
filtro = dfsudeste2010.total_recomendacoes > 0
dfsudeste2010.loc[filtro].groupby(['ocorrencia_cidade']).total_recomendacoes.sum().sort_values(ascending=False)

ocorrencia_cidade
SÃO PAULO                41
RIO DE JANEIRO           25
BRAGANÇA PAULISTA         9
CAMANDUCAIA               9
RESENDE                   8
JUNDIAÍ                   7
IPERÓ                     3
NOVO HORIZONTE            3
TIMÓTEO                   3
BELO HORIZONTE            2
SÃO BERNARDO DO CAMPO     2
BOITUVA                   1
BOM JESUS DO GALHO        1
ITANHAÉM                  1
Name: total_recomendacoes, dtype: int64

In [47]:
#Quantidade de recomendações por mês
dfsudeste2010.loc[filtro].groupby(['ocorrencia_cidade', dfsudeste2010.ocorrencia_dia.dt.month]).total_recomendacoes.sum().sort_values(ascending=False)

ocorrencia_cidade      ocorrencia_dia
RIO DE JANEIRO         8                 19
SÃO PAULO              2                 14
                       12                11
BRAGANÇA PAULISTA      11                 9
CAMANDUCAIA            7                  9
RESENDE                4                  8
SÃO PAULO              1                  8
                       11                 6
JUNDIAÍ                5                  5
RIO DE JANEIRO         4                  3
                       7                  3
TIMÓTEO                2                  3
NOVO HORIZONTE         7                  3
IPERÓ                  1                  3
JUNDIAÍ                6                  2
SÃO BERNARDO DO CAMPO  1                  2
SÃO PAULO              6                  2
BELO HORIZONTE         3                  1
ITANHAÉM               9                  1
BOM JESUS DO GALHO     12                 1
BOITUVA                7                  1
BELO HORIZONTE         2              